In [4]:
import pandas as pd
import ast

In [3]:
df = pd.read_csv('../../data/master.csv')
big_imgs = {}
for i, row in df.iterrows():
    if type(row['images']) == str:
        l = ast.literal_eval(row['images'])
        big_imgs[row['product_id']] = l
        # break
# big_imgs

/tmp/ipykernel_5051/2956587179.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/master.csv')


{'fffb52af-00a7-400c-8f1a-9e79c9fc3d50': ['https://assets.ajio.com/medias/sys_master/root/20201107/3bZw/5fa59a5ff997dd8c838cc2ea/-473Wx593H-460758186-white-MODEL.jpg'],
 'fff7215a-7f9d-4b57-a896-5f364fcf75e6': ['http://assets.myntassets.com/assets/images/8215785/2018/12/19/b12bdd12-5636-46f9-a51e-0af08edf422c1545219718533-Freehand-Men-Blue-Printed-Straight-Kurta-6461545219717832-1.jpg'],
 'fff6f0a1-8292-4782-ba35-3e84037d6d3b': ['https://assets.ajio.com/medias/sys_master/root/20211123/jaY1/619c0991f997ddf8f11390ca/-1117Wx1400H-469009976-black-MODEL.jpg'],
 'ffebf3f0-7cbb-45ee-8323-6a10d3e222c4': ['http://assets.myntassets.com/assets/images/18291354/2022/5/31/7c67bb1c-0e4a-4d4c-bb95-f3ea88e1f4211653980795077-The-Indian-Garage-Co-Men-Jeans-9631653980794436-1.jpg'],
 'ffeb64fb-be8c-4ff6-b6e6-1cdb7a8d90bc': ['https://assets.ajio.com/medias/sys_master/root/20220404/FTwR/624b096df997dd03e246765a/-1117Wx1400H-469100763-cream-MODEL.jpg'],
 'ffd72191-e541-4e83-b896-03d4f5a46f93': ['https://asse

In [4]:
import wget
from urllib.error import HTTPError
for product in big_imgs.keys():
    for i, url in enumerate(big_imgs[product]):
        ext = url.split('.')[-1]
        try:
            wget.download(url,out=product+'#'+str(i)+'.'+ext)
        except:
            pass
    # break

In [8]:
!optimize-images -mw 400 --only-progress ../../data/images_og



Using these symbols:

  ✅ Optimized file     ℹ️  EXIF info present
  🔴 Skipped file       ⤵  Image was downsized     🔻 Size reduction (%)


Recursively searching and optimizing image files in:
../../data/images_og

[62.0s 100.0%] ✅ 10244 🔴 28, saved 605.2 MB
----------------------------------------

   Processed 10272 files (788.5 MB) in 61.5s (166.9 f/s).
   Optimized 10244 files.
   Average savings: 60.5 KB per optimized file
   Total space saved: 605.2 MB / 76.8%



In [11]:
from pathlib import Path
from PIL import Image
paths = Path("../../data/images_og").glob("**/*.jpg")
for path in paths:
    image = Image.open(path)
    destination = path.with_suffix(".webp")
    image.save(destination, format="webp")
    # break

In [1]:
from pathlib import Path

paths = Path("../../data/images_compressed_webp").glob("**/*.webp")
# print(list(paths))
img_changes = {}
for path in paths:
    path = str(path).split('/')[-1]
    id = str(path).split('#')[0]
    path = path.replace('#', '%23')
    path = f'https://tigc.s3.amazonaws.com/images/{path}'
    if not img_changes.get(id):
        img_changes[id] = [path]
    else:
        img_changes[id].append(path)

In [2]:
img_changes

{'c5751fc4-8aa5-4444-8647-63d7f9a5c752': ['https://tigc.s3.amazonaws.com/images/c5751fc4-8aa5-4444-8647-63d7f9a5c752%230.webp'],
 '8af24ca9-95db-4be9-a71b-21deb085a6c6': ['https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%231.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%237.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%232.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%230.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%233.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%239.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%236.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%234.webp',
  'https://tigc.s3.amazonaws.com/images/8af24ca9-95db-4be9-a71b-21deb085a6c6%235.webp',
  'https://tigc.s3.amazonaws.com/images

In [7]:
df = pd.read_csv('../../data/styles.csv')
new_styles = df.copy()
new_styles.set_index('product_id', inplace=True)
for img in img_changes.keys():
    new_styles.at[img,'images'] = img_changes[img]
# new_styles['images']
new_styles.to_csv('../../data/styles_with_new_images.csv')